# Análisis Exploratorio de Datos

## California Housing prices
Se pretende estudiar los datos de un censo de California para construir un modelo que prediga los precios de viviendas del estado.
Estos datos incluyen métricas como la población, el ingreso medio y el precio medio de las viviendas para cada grupo de bloques en California. Los grupos de bloques son la unidad geográfica más pequeña para la cual la Oficina del Censo de los Estados Unidos publica datos (un grupo de bloques típicamente tiene una población de 600 a 3,000 personas). Podemos referirnos a ellos como "distritos" de manera abreviada. El modelo debe aprender de estos datos y ser capaz de predecir el precio medio de las viviendas en cualquier distrito, dado el resto de las métricas.

**¿Qué tipo de entrenamiento necesitamos para nuestro modelo? ¿Qué tarea se va a realizar?**

Esto es claramente una tarea típica de `aprendizaje supervisado`, ya que el modelo puede ser entrenado con datos etiquetados (cada instancia viene con la salida esperada, es decir, el precio medio de las viviendas del distrito). Es una tarea de _regresión_ típica, ya que el modelo predice un valor. Más específicamente, se trata de un problema de _regresión múltiple_, porque el sistema utilizará múltiples características para hacer una predicción (la población del distrito, el ingreso medio, etc.). También es un problema de _regresión univariada_, ya que solo tratamos de predecir un único valor para cada distrito. Si quisieramos predecir múltiples valores por distrito, sería un problema de _regresión multivariada_.

## Data pipeline o flujo de datos

Una secuencia de componentes de procesamiento de datos se llama flujo de datos, _data pipeline_ o simplemente _pipeline_. Los _pipelines_ son muy comunes en sistemas de aprendizaje maquinal, ya que hay muchos datos para manipular y muchas transformaciones de datos que aplicar.

Nosotros vamos a utilizar este concepto de _pipeline_ para transformar los datos, aplicar algún algoritmo de aprendizaje maquinal (modelo) y evaluar posteriormente dicho algoritmo. Particularmente utilizaremos herramientas de la biblioteca de scikit-learn junto con pandas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Cargamos los datos

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/ContenidosPorTemas'
    print('google.colab')

In [ ]:
df_housing = pd.read_csv("./1_datos/housing.csv")
df_housing.head()

Cada fila representa un distrito. La tabla tiene 10 atributos: longitud, latitud, mediana de la edad de viviendas, total de habitaciones, total de dormitorios, población, hogares, mediana del ingreso, mediana del valor de la vivienda y proximidad al océano.

**Para conocer los atributos existentes y el número de instancias pertenecientes a cada atributo**

In [ ]:
df_housing.info()

Hay 20,640 instancias en el conjunto de datos. El atributo total_bedrooms solo tiene 20,433 valores no nulos, significa que faltan 207 distritos en este atributo. Necesitaremos ocuparnos de esto más adelante. Todos los atributos son numéricos, excepto `ocean_proximity`. Su tipo de dato es `object`, por lo que podría contener cualquier tipo de dato de Python. Pero al cargar los datos desde un archivo CSV, sabemos que debe ser texto. Al observar las cinco primeras filas, notamos que los valores en la columna ocean_proximity son repetitivos, lo que sugiere que probablemente es un _atributo categórico_.

**Para conocer las categorias existentes y el número de instancias pertenecientes a cada categoría**

In [ ]:
df_housing["ocean_proximity"].value_counts()

**describe() muestra un resumen de los atributos numéricos**

In [ ]:
df_housing.describe()

**Un percentil indica el valor por debajo del cual cae un cierto porcentaje de observaciones o instancias en un grupo de observaciones.**

In [ ]:
df_housing.hist(bins=50, edgecolor='k', figsize=(20,15))
plt.show()

- El atributo de `median_income` no parece estar expresado en USD. Después de verificar, informan que los datos se han escalado y limitado a 15 para `median_income` altos, y a 0.5 para `median income` más bajos. Los números representan aproximadamente decenas de miles de dólares (por ejemplo, 3 en realidad significa alrededor de \$30,000). Trabajar con atributos preprocesados es común en aprendizaje automático, y no necesariamente es un problema, pero se debe comprender cómo se calcularon los datos.
- Los atributos `housing_median_age` y `median_house_value` también fueron limitados. Este último podría ser un problema grave ya que es el atributo objetivo o _target_ (etiqueta). El algoritmo de aprendizaje automático podría aprender que los precios nunca superan ese límite. Si nuestro cliente nos dice que necesita predicciones precisas incluso más allá de $500,000, entonces tenemos dos opciones:
    - Recopilar etiquetas adecuadas para los distritos cuyas etiquetas fueron limitadas.
    - Eliminar esos distritos del conjunto de datos.
- Algunos histogramas están sesgados hacia la derecha: se extienden mucho más hacia la derecha de la mediana que hacia la izquierda. Esto puede dificultar un poco la detección de patrones para algunos algoritmos de aprendizaje maquinal.

## Advertencia ⚠️

Antes de continuar observando los datos, vamos a separar un conjunto de prueba y no lo miramos más!!!

Si se observa el conjunto de prueba, es posible encontrar algun patrón interesante en los datos que nos lleve a seleccionar un tipo particular de modelo de aprendizaje maquinal. Al evaluar el modelo utilizando el conjunto de prueba, la estimación será demasiado optimista y el sistema no funcionará tan bien como se espera. A esto se llama sesgo de búsqueda de datos (data snooping bias).

## Creamos un conjunto de prueba

**¿Qué es un muestreo estratificado?**

Supongamos que conversamos con expertos que nos dicen que el `median_income` es un atributo muy importante para predecir la mediana de precios de las viviendas. Debemos asegurar que el conjunto de prueba sea representativo de las categorías de ingresos en el conjunto de datos. Dado que el `median_income` es un atributo numérico continuo, primero necesitamos crear un atributo de categoría de ingresos `income_cat`.

Vamos a observar más de cerca el histograma de `median_income`:

In [ ]:
_ = df_housing["median_income"].hist(bins=50, color='skyblue', edgecolor='k', figsize=(7,5))

la mayoría de los valores de ingresos se agrupan alrededor de 1.5 a 6 (es decir, \$15,000 - \$60,000), pero algunos van mucho más allá de 6. Es importante tener un número suficiente de instancias en tu conjunto de datos para cada estrato o categoría, o de lo contrario la estimación de la importancia de un estrato podría estar sesgada. Esto significa que no debemos tener demasiados estratos, y cada estrato debe ser lo suficientemente grande.

El siguiente código utiliza la función `pd.cut()` para crear un atributo `income_cat` con cinco categorías (etiquetadas del 1 al 5), la categoría 1 abarca desde 0 hasta 1.5 (es decir, menos de $15,000), la categoría 2 desde 1.5 hasta 3, y así sucesivamente:

In [ ]:
df_housing["income_cat"] = pd.cut(df_housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])
df_housing.head()

In [ ]:
df_housing["income_cat"].value_counts().sort_index().plot.bar(rot=0)
plt.xlabel("Income category")
plt.ylabel("Number of districts")
_ = plt.grid()

Ahora podemos hacer un muestreo estratificado con la categoría `income_cat`

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df_housing, test_size=0.2, stratify=df_housing["income_cat"], random_state=42)

In [ ]:
# Eliminamos la categoria income_cat de ambos conjuntos porque no la usamos más
for set_ in (train_set, test_set):
    set_.drop("income_cat", axis=1, inplace=True)

## Visualización del conjunto de entrenamiento

In [ ]:
# Antes de continuar hacemos una copia del conjunto de entrenamiento
housing = train_set.copy()

### Visualización geográfica

In [ ]:
housing.plot(kind = "scatter", x = "longitude", y = "latitude", alpha=0.2, figsize=(10,7))
plt.show()

In [ ]:
housing.plot(kind="scatter", x = "longitude", y = "latitude", alpha=0.4,
              s = housing["population"]/100 , label="población", c="median_house_value",
              cmap="jet", colorbar = True, legend = True, sharex = False, figsize=(10,8))
plt.show()

Esta imagen indica que los precios de las viviendas están muy relacionados con la ubicación (por ejemplo, cerca del océano) y con la densidad de población.

### Correlaciones

Podemos calcular facilmente el _índice de correlación de Pearson_ entre cada par de atributos.

In [ ]:
corr_matrix = housing.corr(numeric_only=True)
corr_matrix

Y la correlación de cada atributo con `median_house_value`

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

El coeficiente de correlación varía de -1 a 1. Cuando está cerca de 1, significa que hay una fuerte correlación positiva; por ejemplo, el `median_house_value` tiende a aumentar cuando el `median_income` aumenta. Cuando el coeficiente está cerca de -1, significa que hay una fuerte correlación negativa; se puede observar una pequeña correlación negativa entre la latitud y el `median_house_value` (es decir, los precios tienen una ligera tendencia a disminuir cuando nos dirigimos al norte). Finalmente, los coeficientes cercanos a 0 significan que no hay una correlación lineal.

In [ ]:
from pandas.plotting import scatter_matrix

atributos = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[atributos], figsize=(15,10), hist_kwds={'bins':50, 'color':'salmon', 'edgecolor':'k'})
plt.show()

## Resumen de la primera parte: Análisis exploratorio de datos

En esta primera parte:
- Obtuvimos los datos
- Hicimos una exploración inicial para determinar:
    - número de instancias
    - número de atributos
    - tipos de datos de los atributos
    - si hay datos faltantes
- Dividimos los datos en conjuntos de entrenamiento y prueba. La división se hizo de forma estratificada en función de un atributo definido por expertos.
- Exploramos más en profundidad el conjunto de entrenamiento mediante visualizaciones, búsqueda de atributos correlacionados.
